# Spark: Dataframes y Spark SQL. 

Los dataframes se obtienen a partir de RDDs. 
Se pueden ver como RDDs de tipo *Row*

Un ejemplo está tomado de https://www.codementor.io/jadianes/python-spark-sql-dataframes-du107w74i

Muchos ejemplos de estadísticas y SparkSQL:

https://github.com/databricks/Spark-The-Definitive-Guide/blob/master/code/Structured_APIs-Chapter_7_Aggregations.py

# 0 Instalación

Ejecutar para asegurar de que se tienen las librerías adecuadas

In [1]:
modules = ["findspark","pyspark","pyspark_dist_explore"]

import sys
import os.path
from subprocess import check_call
import importlib
import os

if 'google.colab' in sys.modules:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    spark = "spark-3.2.0-bin-hadoop3.2.tgz"
    if not os.path.isfile(spark):
        !wget -q https://downloads.apache.org/spark/spark-3.2.0/{spark}
        !tar xf {spark}
        os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
        os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

print("Checking required modules")
for m in modules:
    torch_loader = importlib.util.find_spec(m)
    if torch_loader is not None:
        print(m," found")
    else:
        print(m," not found, installing")
        if 'google.colab' in sys.modules:
            check_call(["pip", "install", "-q", m])
        else:
            check_call([sys.executable, "-m", "pip", "install", "--user", m])
print("Done!")
#!pip install -q findspark
#  !pip install -q pyspark
print("Ready!")

Checking required modules
findspark  found
pyspark  found
pyspark_dist_explore  found
Done!
Ready!


**OJO** El código a continuación no es necesario en el laboratorio, ni en colaborate, ni tampoco si habéis añadido las variables de entorno desde fuera de Jupyter. Lo dejo como texto, convertirlo a código si se necesita

In [2]:

    # descargar la carpeta copiar la carpeta spark en c:\hlocal\tdm
    import os
    # cambiamos las variables del sistema
    spark = 'C:/hlocal/tdm/spark/spark-3.1.2-bin-hadoop3.2'

    if not(os.path.isdir(spark+"/bin")) or not(os.path.isdir(spark+"/jars")) :
        print("Error, la carpeta en 'spark' debe contener los directorios bin y jars ")
    else:    
        # en el path se añade
        #path = os.environ.get('PATH') 
        #path = path+ ';'+spark+'\\bin;'
        #os.environ['PATH'] = path
        os.environ['SPARK_HOME']= spark 
        os.environ['HADOOP_HOME']= spark 
        os.environ['PYSPARK_DRIVER_PYTHON']= 'jupyter'
        os.environ['PYSPARK_DRIVER_PYTHON_OPTS']='notebook'

        # si da problema con collect quizás haya que poner java_home a la localización de java 8
        #os.environ['JAVA_HOME']= 'C:\\Program Files\\Java\\jdk1.8.0_151'
        #os.environ['PATH'] = os.environ.get('JAVA_HOME')+'\\bin;'+spark
        print("Hecho")
            

Hecho


## Comenzamos

Iniciamos Spark

In [4]:


import findspark
findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext


df = spark.sql('''select 'spark' as hola ''')
df.show()

#df = sc.parallelize([1,2,3,4,5,6],1)
#df.collect()




+-----+
| hola|
+-----+
|spark|
+-----+



### Carga de ficheros desde Spark; paso de RDD a DataFrame

Siempre podemos crear un DataFrame "manualmente" siguiendo estos pasos:


1. Crear/cargar un RDD con los datos que queramos (un RDD de tuplas o de listas)

2. Usar *map* para convertir el RDD en un RDD de *Rows*

3. Utilizar *spark.createDataFrame*

Una posibilidad alternativa, y por supuesto preferible siempre y cuando el formato de datos dde entrada lo perrmita, es cargar directamente el fichero en formato DataFrame

En el siguiente ejemplo, partimos de un fichero destinado a detectar ataques a partir de datos de entradas en logs (ver [https://kdd.ics.uci.edu/databases/kddcup99/task.html](https://kdd.ics.uci.edu/databases/kddcup99/task.html))

In [5]:
# Alternatuva 1: construcción manual

import urllib.request
url = "http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz"
path = "./kddcup.gz"
f = urllib.request.urlretrieve(url,path )



# descargar de 
# http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz
# pero de momento leemos un RDD 
raw_data = sc.textFile(path).cache() # Un RDD de strings
raw_data.take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 3) (DESKTOP-TLRRHMV executor driver): java.net.SocketException: Connection reset by peer: socket write error
	at java.net.SocketOutputStream.socketWrite0(Native Method)
	at java.net.SocketOutputStream.socketWrite(Unknown Source)
	at java.net.SocketOutputStream.write(Unknown Source)
	at java.io.BufferedOutputStream.flushBuffer(Unknown Source)
	at java.io.BufferedOutputStream.write(Unknown Source)
	at java.io.DataOutputStream.write(Unknown Source)
	at java.io.FilterOutputStream.write(Unknown Source)
	at org.apache.spark.api.python.PythonRDD$.writeUTF(PythonRDD.scala:477)
	at org.apache.spark.api.python.PythonRDD$.write$1(PythonRDD.scala:297)
	at org.apache.spark.api.python.PythonRDD$.$anonfun$writeIteratorToStream$1(PythonRDD.sca

Ahora pasamos de RDD a Dataframe

In [ ]:
from pyspark.sql import Row

csv_data = raw_data.map(lambda l: l.split(",")) 
csv_data.take(10)

In [ ]:
row_data = csv_data.map(lambda p: Row(
    duration=int(p[0]), 
    protocol_type=p[1],
    service=p[2],
    flag=p[3],
    src_bytes=int(p[4]),
    dst_bytes=int(p[5]),
    wrong_fragment=int(p[7])
    )
)
row_data.take(5)

In [ ]:

# creamos el dataframe
interactions_df = spark.createDataFrame(row_data)


interactions_df.show()

In [ ]:

# Load  data
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/escalado.csv"
file = "./escalado.csv"
f = urllib.request.urlretrieve(url,file )
datos = spark.read.format("com.databricks.spark.csv")\
            .options(header='true', inferschema='true') \
            .load(file)
datos.show()

Mejor ver solo es esquema:

In [ ]:
datos.printSchema()

In [ ]:
interactions_df.printSchema()

### SparkSQL

Una vez que tenemos un DataFrame podemos consultarlo de forma más cómoda que con las acciones y transformaciones de los RDDs

In [ ]:
interactions_df.registerTempTable("interactions") # de esta forma se puede referir a la tabla interactions

tcp_interactions = spark.sql("""
    SELECT duration, dst_bytes FROM interactions 
    WHERE protocol_type = 'tcp' AND 
          duration > 1000 AND 
          dst_bytes = 0
""")
tcp_interactions.show()

También se puede usar adaptaciones de SQL sin tener que entrecomillar, nosotros usaremos algunos conceptos sencillos. Comenzamos por seleccionar unas pocas columnas:

In [ ]:
names = ["duration", "dst_bytes"]
interactions_df.select(names).show()


Ahora contamos valores no nulos de una columna

In [ ]:
from pyspark.sql.functions import count,countDistinct,min,max,sum

# Esto es un poco tonto es contar el número de filas si service nunca es nulo
interactions_df.select(count("service")).show() # 494021; e

In [ ]:
interactions_df.select(max("duration")).show() 

In [ ]:
# número de valores diferentes que toma 'service'
interactions_df.select(countDistinct("service")).show() # 66

In [ ]:
# cuales son los valores distintos?
interactions_df.select("service").distinct().show()

### Estadísticas descriptivas

Comenzamos por ver datos básicos, como la media, desviación típica y valores máximos y mínimos

In [ ]:
# spark
#datos.describe().show() 

# usando pandas; ojo utilizar sample
datos.describe().toPandas().transpose()

In [ ]:
interactions_df.describe().toPandas().transpose()

Los histogramas pueden ayudar a entender mejor los datos. Se hacen para cada variable por separado

In [ ]:
from pyspark_dist_explore import hist
import matplotlib.pyplot as plt
%matplotlib inline

fig,ax = plt.subplots()
col = 'long_media_mensajes'
hist(ax, datos.select([col]), bins = 20, color=['red'])
plt.xlabel(col)
plt.show()

En ocasiones se entiende mejor si se aplica alguna función. En este caso tiene un tipo exponencial, 
así que parece que un logaritmo nos puede ayudar a entender mejor el dato

In [ ]:
import math

# devuelve un valor tipo Row con el logaritmo del valor
def applyLog(x):
    if x[0]==0:
        v=0
    else:
        v=math.log(x[0])
    return Row(lx=v)



fig,ax = plt.subplots()
col = 'long_media_mensajes'
d = spark.createDataFrame(datos.select([col]).rdd.map(applyLog))
hist(ax, d, bins = 20, color=['blue'])
plt.xlabel('log('+col+')')
plt.show()

Parece una normal muy abultada. Para comprobarlo podemos acceder a dos parámetros tipicos, curtosis y asimetría:

https://support.minitab.com/es-mx/minitab/18/help-and-how-to/statistics/basic-statistics/supporting-topics/data-concepts/how-skewness-and-kurtosis-affect-your-distribution/


In [ ]:
from pyspark.sql.functions import skewness, kurtosis
d.select(skewness("lx"), kurtosis("lx")).show()

En una normal perfecta skewness es 0. Valores negativos indican asimetría a la izquierda y positivos a la derecha (o positiva). 

Igualmente, la curtosis sería 0 (o 3, depende de la librería). Un valor positivo indica Leptocúrtica o apuntada y un valor negativo platicúrtica o "achatada"

Esto indica que tenemos asimetría positiva y leptocúrtica (apuntada). Para ver como son estos valores para una distribución normal en esta librería:

In [ ]:
from pyspark.sql.functions import rand, randn

d2 = d.select(randn().alias("normal"))

fig,ax = plt.subplots()
col = 'normal'
hist(ax, d2.select([col]), bins = 20, color=['red'])
plt.xlabel(col)
plt.show()

d2.select(skewness("normal"), kurtosis("normal")).show()

### Correlaciones

Muy importantes porque son columnas que a menudo hay que eliminar

In [ ]:
from pyspark.sql.functions import corr
datos.select(corr("num_likes_com", "conv_insultos")).show()

In [ ]:
for i in range(len(datos.columns)):
    for j in range(len(datos.columns)):
        c = datos.select(corr(datos.columns[i], datos.columns[j])).first()
        if math.fabs(c[0])>0.65 and i<j:
            print(c)

### Muestras

Nos interesa representar gráficamente la relación entre *long_media_debates_com* y *long_media_debates_otra*. Sin embargo, en un entorno big data no tiene sentido mostrar todos los puntos. En lugar de eso tomamos una muestra aleatoria. Para ello usaremos *sample*, que recibe tres parámetros:


    * withReplacement = True o False según se desee una muestra con o sin reemplazamiento.
    * fraction, valor entre 0 y uno con la proporción de la población a seleccionar. 
    * seed : semilla aleatorio si se quiere que el resultado sea reproducible 



In [ ]:
import matplotlib.pyplot as plt
n = datos.count()
campos = ['long_media_debates_com','long_media_debates_otra']
d = datos.select(campos).sort(campos[0])
puntos = 100 # cuántos puntos deseamos
if n<puntos:
    p = 1 # seleccionar todos
else:
    p = puntos / n
puntos = d.sample(False,p).collect()

# mostrar
plt.scatter(list(map(lambda x:x[0],puntos)),list(map(lambda x:x[1],puntos)), color='darkgreen', label=campos[0])
#plt.scatter(list(map(lambda x:x[1],puntos)), color='darkred', label=campos[1])
plt.show()


Los valores 0 "molestan". ¿Podriamos quitarlos? ¿Antes o después del *sample*?

In [ ]:
d.registerTempTable("tablagraf") # de esta forma se puede referir a la tabla interactions

positivos = spark.sql("""
    SELECT * FROM tablagraf 
    WHERE long_media_debates_com > 0.1 AND long_media_debates_otra > 0.1
""")
positivos.show()

In [ ]:
puntos = 100 # cuántos puntos deseamos
n = positivos.count()
if n<puntos:
    p = 0.99 # seleccionar todos
else:
    p = puntos / n
    
print(p)    
puntos = positivos.sample(False,p).collect()

# mostrar
plt.scatter(list(map(lambda x:x[0],puntos)),list(map(lambda x:x[1],puntos)), color='darkgreen', label=campos[0])
#plt.scatter(list(map(lambda x:x[1],puntos)), color='darkred', label=campos[1])
plt.show()

#### ¿Cómo calcular la recta que mejor representa estos puntos? ==> regresión (siguiente tema)